# Human Protein Atlas Featurization


In this notebook, we explore the Human Protein Atlas Dataset and discuss the strategy we used to create featurizers using this dataset.

## The Dataset

The dataset that we used for our HPA featurizers is from the [Kaggle Human Protein Atlas Image Classification Challenge](https://www.kaggle.com/c/human-protein-atlas-image-classification). The dataset contains a large number of confocal microscopy imaging of cells. Each sample is represented by four channels: nucleus (blue), microtubules (red), endoplasmic reticulum (yellow) and protein (green). Each sample is also paired with 27 different cell type labels, but for our purpose, only the images were used.

## Imports

In [1]:
%matplotlib inline
%gui qt5
import napari
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

## Example from the dataset

The four different channels represent: 
- Microtubules (Red)
- Protein (Green)
- Nucleus (Blue)
- Endoplasmic Reticulum (Yellow)

In [7]:
image = np.load("hpa_example.npy")
viewer = napari.view()
viewer.add_image(image[0], colormap ="red", name="Microtubules", blending="additive")
viewer.add_image(image[1], colormap ="green", name="Protein", blending="additive")
viewer.add_image(image[2], colormap ="blue", name="Nucleus", blending="additive")
viewer.add_image(image[3], colormap ="yellow", name="ER",  blending="additive")

<Image layer 'ER' at 0x7f4ce65c6c88>

## Data Processing

After downloading the dataset from Kaggle, we processed and stored all training examples into a HDF5 file. The code we used to generate this file can be found [here](https://github.com/marshuang80/BioSegmentation/blob/master/process_data/hpa_create_hdf5.py)

You can specify the input folder of your data and your desired output directory as the following:

```
python create_hpa_hdf5.py --input_dir "/home/user/hpa/" \
                          --output_dir "/home/user/data/"
```

After the HDF5 file is generated, we impletemented a PyTorch Dataset to process these input data for the model [code](https://github.com/marshuang80/BioSegmentation/blob/master/dataset/hpa_dataset.py)

## Training

To train our featurizer, we have implemented a simple [UNet model](https://arxiv.org/pdf/1505.04597)

![](./figs/unet.png)

The impletmentation code can be found [here](https://github.com/marshuang80/BioSegmentation/blob/master/model/unet.py)

Since segmentify currently only supports black and white images (one color channel) and HPA images have four channels, we can either take the max or mean across the color channels and use the resulting image as input: 


| Max | Mean | 
| --- | --- |
| ![](figs/hpa_max.png) |  ![](figs/hpa_mean.png) |

The choice of mean or max depends on the segmentation target. We have experimented with several output targets:

- 1 channel (nucleus channel)
- 3 channels (nucleus, microtubles, ER)
- 4 channels (nucleu, microtubles, ER and protein)



**NEED EDIT**

The target channels can be binarized with a threshold (0.5) or leave as original. 

To train the model, run **train.py** from [BioSegmentation](https://github.com/marshuang80/BioSegmentation) with the right parameters, for example: 

```
python train.py --num_kernel 8 \
                --kernel_size 3\
		        --lr 1e-3 \
		        --epoch 200\
			    --train_data /home/user/Nuclei/train.hdf5 \
			    --val_data /home/user/Nuclei/val.hdf5 \
			    --save_dir ./ \
                --device cuda\
                --optimizer adam\
                --model unet\
                --shuffle False \
                --num_workers 16 \
                --vflip False \
                --hflip False \
                --zoom True \
                --rotate False \
                --batch_size 64 \
                --gpu_ids 0,1,2,3\
                --experiment_name unet_k8_s3_adam
```

The saved model is called **UNet.pth**

## Featurize

After the segmentation network is trained and saved, we can then use the trained model as a featurizer by removing the last layer or simply switch the last layer to an identity function. The featurizing code can be found ([here](https://github.com/transformify-plugins/segmentify/blob/master/segmentify/semantic/main.py))

The different featurized image can be visualized with the following block:

In [33]:
# 1 channel (nucleus)
nuclei_features = np.load("hpa_1c_max_features.npy")
nuclei_features = np.transpose(nuclei_features, (2,0,1))
viewer = napari.view(nuclei_features)

In [37]:
# 3 channel (nucleus, microtubles, ER)
nuclei_features = np.load("hpa_3c_mean_features.npy")
nuclei_features = np.transpose(nuclei_features, (2,0,1))
viewer = napari.view(nuclei_features)

In [36]:
# 4 channel (nucleus, microtubles, ER, protein)
nuclei_features = np.load("hpa_4c_mean_features.npy")
nuclei_features = np.transpose(nuclei_features, (2,0,1))
viewer = napari.view(nuclei_features)

## Results

We use the following steps to evaluate our featurizers:
- Featurize all images (including train and test)
- Pick 20% of the pixels from the training data
- Train a Random Forest Classifier using the 20% selected pixels
- Predict binary segmentation on test set using trained RFC/
- Remove small islands

Using the 3 different HPA featurizers, we are able to obtain the results below. The x axis show the number of training examples used, and the y axis is the performance metric (IoU, precision).


**1 channel (nucleus) max 8 dimentions**:
![](./figs/hpa_1c_8_max.png)

**3 channel (nucleus) mean 16 dimentions**:
![](./figs/hpa_3c_16_mean.png)

**4 channel (nucleus) mean 16 dimentions**:
![](./figs/hpa_4c_16_mean.png)